In [3]:
import pandas as pd
import plotly.express as px

df = pd.read_csv("../../../data/ex1/preprocessed_ex2.csv")
df.head()

,day,hour,ICAO,OnGround
0,01/12/2024,0,02012B,0.0
1,01/12/2024,0,020176,0.0
2,01/12/2024,0,02019D,0.0
3,01/12/2024,0,342352,1.0
4,01/12/2024,0,342353,1.0


***Gráfico de barras del tráfico aéreo por horas, distinguiendo aviones en tierra y en el aire***

In [2]:
all_hours = pd.DataFrame({"hour": list(range(24))})

days = df["day"].unique()
for day in days:
    df_day = df[df["day"] ==day]
    df_day =df_day.groupby(["hour","OnGround"])["OnGround"].size().reset_index(name="count")
    df_day = all_hours.merge(df_day, on="hour", how="left").fillna(0)
    df_day["OnGround"] = df_day["OnGround"].replace({1: "On Ground", 0: "In Air"})
    
    fig = px.bar(df_day, x="hour", y="count", color="OnGround", barmode="stack")

    fig.update_layout(
        title =dict(
            text = f"Day {day}",
            x = 0.48,
            font = dict(
                size = 20,
                color = "black",
                weight = "bold"
            )
        ),
    xaxis=dict(
        title='Horas',
        tickfont=dict(
            size=12,
            color='black',
        ),
        tickmode="linear",
        dtick=1
    ),
    yaxis= dict(
        title = "Frecuencia",
        tickfont=dict(
            size=12,
            color='black',
        )
    )
    )
    
    fig.show()
    

In [3]:
import plotly.graph_objects as go
import pandas as pd
import math

all_hours = pd.DataFrame([
    {"hour": h, "OnGround": og} 
    for h in range(24) for og in [0, 1]
])


days = df["day"].unique()

max_y = 0  
for day in days:
    df_day = df[df["day"] == day]
    df_day = df_day.groupby(["hour", "OnGround"])["OnGround"].size().reset_index(name="count")
    df_day = all_hours.merge(df_day, on=["hour", "OnGround"], how="left").fillna(0)
    
    max_y = max(max_y, df_day["count"].max())

max_y = math.ceil(max_y / 10) * 10

for day in days:
    df_day = df[df["day"] == day]
    
    df_day = df_day.groupby(["hour", "OnGround"])["OnGround"].size().reset_index(name="count")
    
    df_day = all_hours.merge(df_day, on=["hour", "OnGround"], how="left").fillna(0)

    on_ground_count = df_day[df_day["OnGround"] == 1]["count"].values
    flying_count = df_day[df_day["OnGround"] == 0]["count"].values
    hours = df_day[df_day["OnGround"] == 1]["hour"].values  
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hours, y=on_ground_count, mode='lines+markers', name='En Tierra'))
    fig.add_trace(go.Scatter(x=hours, y=flying_count, mode='lines+markers', name='En Aire'))

    fig.update_layout(
        title=f"Tráfico Aéreo por Hora - Día {day}",
        xaxis_title="Hora del Día",
        yaxis_title="Cantidad de Aviones",
        xaxis=dict(tickmode="linear", dtick=1),
        yaxis=dict(range=[0, max_y]), 
        template="plotly_dark"
    )

    fig.show()


***Cantidad de aviones en tierra y aire por horas*** (en diferentes graficos)

In [ ]:
all_days = pd.DataFrame([
    {"day": d, "hour": h} 
    for d in days for h in range(24)
])
df_merged = all_days.merge(df, on=["day","hour"], how="left").fillna(0)
# Contar el número de aeronaves en tierra y en aire por hora y día
ground_count = df_merged[df_merged["OnGround"] == 1].groupby(["day", "hour"]).size().reset_index(name="count")
air_count = df_merged[df_merged["OnGround"] == 0].groupby(["day", "hour"]).size().reset_index(name="count")

# Gráfico para aviones en tierra
fig_ground = px.line(ground_count, x="hour", y="count", color="day",
                     labels={"hour": "Hora del día", "count": "Cantidad de aeronaves", "day": "Día"})

# Gráfico para aviones en aire
fig_air = px.line(air_count, x="hour", y="count", color="day",
                  labels={"hour": "Hora del día", "count": "Cantidad de aeronaves", "day": "Día"})

fig_ground.update_layout(
     xaxis=dict(tickmode="linear", dtick=1),
     title=dict(       
            text = "Cantidad de aviones en tierra por hora",
            x = 0.5,
            font=dict(
                family="Arial Black, sans-serif",
                size=18,
                color="black"
            )
        ),
)
fig_air.update_layout(
     xaxis=dict(tickmode="linear", dtick=1),
     title=dict(       
            text = "Cantidad de aviones en aire por hora",
            x = 0.5,
              font=dict(
                family="Arial Black, sans-serif",
                size=18,
                color="black"
            )
        ),
)

# Mostrar los gráficos
fig_ground.show()
fig_air.show()

***Matriz de calor del tráfico aéreo por día y hora***

In [ ]:
df_heatmap = df.groupby(['day', 'hour']).size().reset_index(name='count')
fig = px.imshow(df_heatmap.pivot(index='day', columns='hour', values='count').fillna(0), 
                labels=dict(color="Cantidad de aviones"),
                title="Matriz de calor del tráfico aéreo por día y hora",
                aspect="auto")

fig.update_layout(
        xaxis=dict(tickmode="linear", dtick=1),
        title=dict(       
            text = "Matriz de calor del tráfico aéreo por día y hora",
            x = 0.5,
            font=dict(
                family="Arial Black, sans-serif",
                size=18,
                color="black"
            )
        ),
        xaxis_title = "Horas",
        yaxis_title = "Días",
     
    )
fig.show()

***Porcentaje de aviones en tierra y en el aire por hora***

In [23]:
df_hour = df.groupby(['hour', 'OnGround']).size().reset_index(name='count')
df_hour["OnGround"] = df_hour["OnGround"].replace({1: "On Ground", 0: "In Air"})
df_hour['total'] = df_hour.groupby('hour')['count'].transform('sum')
df_hour['percentage'] = df_hour['count'] / df_hour['total'] * 100

fig = px.area(df_hour, x="hour", y="percentage", color="OnGround", title="Porcentaje de aviones en tierra y en el aire por hora")
fig.update_layout(
      title=dict(
            text="Porcentaje de aviones en tierra y en el aire por hora",
            x=0.5,
            font=dict(
                family="Arial Black, sans-serif",
                size=18,
                color="black"
            )
        ),    
        xaxis_title = "Horas",
        yaxis_title = "Porcentaje",
        xaxis=dict(tickmode="linear", dtick=1)
)
fig.show()

***Tráfico Aéreo por dia de la semana descendiente***

In [20]:

df_day =df.groupby(['day', 'OnGround']).size().reset_index(name='count')

df_day["OnGround"] = df_day["OnGround"].replace({1: "On Ground", 0: "In Air"})

df_day = df_day.sort_values(by="count",ascending=False)

fig = px.bar( df_day,x="day", y="count", color="OnGround", barmode="stack")


fig.update_layout(
        xaxis_title="Día de la semana",
        yaxis_title="Cantidad de Aviones",
        title=dict(
            text="Tráfico Aéreo por dia de la semana",
            x=0.5,
            font=dict(
                family="Arial Black, sans-serif",
                size=18,
                color="black"
            )
        ),
    )